### Imports

In [52]:
import os
import requests
import pdfplumber
import re
import pandas as pd

### Paths

In [53]:
CWD = os.path.join(os.getcwd())
SOURCE_PATH = os.path.join(CWD, 'csv', 'gesetze.csv')
TARGET_PATH = os.path.join(CWD, 'csv', 'gesetze_mit_drucksachen.csv')
DRUCKSACHE_PATH = os.path.join(CWD, 'csv', 'drucksachen_details.csv')
DRUCKSACHE_WITH_DESC_PATH = os.path.join(CWD, 'csv', 'drucksachen_details_with_desc.csv')
API_KEY = 'I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21'

### Load data

In [54]:
df = pd.read_csv(SOURCE_PATH)
df.head()

,Unnamed: 0,date,title,id,topic,url,drucksache
0,0,2024-12-19,Gesetzentwurf zur Änderung von Artikel 93 und ...,939,Geschäftsordnung,/parlament/plenum/abstimmung/abstimmung?id=939,['https://dserver.bundestag.de/btd/20/129/2012...
1,1,2024-12-16,Antrag gemäß Artikel 68 des Grundgesetzes (Ver...,938,Parlament,/parlament/plenum/abstimmung/abstimmung?id=938,['https://dserver.bundestag.de/btd/20/141/2014...
2,2,2024-10-18,Änderungsantrag zur „Verbesserung der inneren ...,929,Inneres,/parlament/plenum/abstimmung/abstimmung?id=929,['https://dserver.bundestag.de/btd/20/128/2012...
3,3,2024-10-18,Artikel 5 des Entwurfs „Verbesserung der inner...,930,Inneres,/parlament/plenum/abstimmung/abstimmung?id=930,['https://dserver.bundestag.de/btd/20/128/2012...
4,4,2024-10-18,Gesetzentwurf „Verbesserung der Terrorismusbek...,931,Inneres,/parlament/plenum/abstimmung/abstimmung?id=931,['https://dserver.bundestag.de/btd/20/128/2012...


### API Key

In [55]:
API_KEY = 'I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21' 

### Get Drucksache JSON functions

In [56]:
def extract_document_number(url):
    parts = url.split('/')
    wahlperiode = parts[4]  # nach "btd/"
    pdf_name = parts[-1].replace('.pdf', '')
    dokumentnummer = pdf_name[len(wahlperiode):]
    return f"{wahlperiode}/{dokumentnummer}"

In [57]:
def get_document_info(doc_numbers):
    api_url = f"https://search.dip.bundestag.de/api/v1/drucksache-text?f.dokumentnummer={doc_numbers}&format=json&apikey={API_KEY}"
    response = requests.get(api_url, headers={'accept': 'application/json'})
    if response.status_code == 200:
        data = response.json()
        if data["numFound"] > 0:
            documents = data["documents"]
            doc_info = {}
            for doc in documents:
                doc_nr = doc["dokumentnummer"]
                doc_id = doc["id"]
                doc_type = doc.get("drucksachetyp", None)
                doc_text = doc.get("text", None)
                doc_info[doc_nr] = {"id": doc_id, "type": doc_type, "text": doc_text}
            return doc_info
    return {}

In [58]:
def process_csv(input_file, output_file, drucksache_output, num_samples=None):
    df = pd.read_csv(input_file)
    if num_samples is not None:
        df = df.head(num_samples)
    df['drucksache_referenz'] = None
    drucksache_data = []
    for i, row in df.iterrows():
        try:
            urls = eval(row['drucksache'])
            referenzen = []
            doc_numbers = [extract_document_number(url) for url in urls]
            doc_info = get_document_info('&f.dokumentnummer='.join(doc_numbers))
            for url, doc_number in zip(urls, doc_numbers):
                doc_data = doc_info.get(doc_number, {})
                drucksache_data.append({
                    "referenznummer": doc_data.get("id", None),
                    "docNr": doc_number,
                    "docTyp": doc_data.get("type", None),
                    "docText": doc_data.get("text", None)
                })
                referenzen.append(doc_data.get("id", None))
            df.at[i, 'drucksache_referenz'] = str(referenzen)
        except Exception:
            continue
    df.to_csv(output_file, index=False)
    drucksache_df = pd.DataFrame(drucksache_data)
    drucksache_df.to_csv(drucksache_output, index=False)

### Create CSVs

In [59]:
url = 'https://dserver.bundestag.de/btd/20/143/2014302.pdf'
docNr = extract_document_number(url)
print(docNr)

doc_info = get_document_info(docNr)
print(doc_info)

20/14302
{'20/14302': {'id': '277864', 'type': 'Beschlussempfehlung und Bericht', 'text': 'Deutscher Bundestag Drucksache 20/14302 \n20. Wahlperiode 18.12.2024 \nBeschlussempfehlung und Bericht \ndes Rechtsausschusses (6. Ausschuss) \na) zu dem Gesetzentwurf der Fraktionen SPD, CDU/CSU, BÜNDNIS 90/DIE\nGRÜNEN und FDP sowie des Abgeordneten Stefan Seidler\n– Drucksache 20/12977 –\nEntwurf eines Gesetzes zur Änderung des Grundgesetzes \n(Artikel 93 und 94) \nb) zu dem Gesetzentwurf der Fraktionen SPD, CDU/CSU, BÜNDNIS 90/DIE\nGRÜNEN und FDP sowie des Abgeordneten Stefan Seidler\n– Drucksache 20/12978 –\nEntwurf eines Gesetzes zur Änderung des  \nBundesverfassungsgerichtsgesetzes und des \nUntersuchungsausschussgesetzes \nA. Problem\nZu Buchstabe a)\nDie den Gesetzentwurf vorlegenden Fraktionen und der Abgeordnete Stefan\nSeidler weisen darauf hin, dass sich das Grundgesetz (GG) für eine starke\nVerfassungsgerichtsbarkeit entschieden habe. Diese zentrale Weichenstellung lasse sich \nan de

In [60]:
process_csv(SOURCE_PATH, TARGET_PATH, DRUCKSACHE_PATH, num_samples=30) # None


### Show CSV results

In [87]:
print("CSV: gesetze_mit_drucksachen.csv")
df_gesetze_mit_drucksachen = pd.read_csv(TARGET_PATH)
df_gesetze_mit_drucksachen.head()

CSV: gesetze_mit_drucksachen.csv


,Unnamed: 0,date,title,id,topic,url,drucksache,drucksache_referenz
0,0,2024-12-19,Gesetzentwurf zur Änderung von Artikel 93 und ...,939,Geschäftsordnung,/parlament/plenum/abstimmung/abstimmung?id=939,['https://dserver.bundestag.de/btd/20/129/2012...,"['276076', '277864']"
1,1,2024-12-16,Antrag gemäß Artikel 68 des Grundgesetzes (Ver...,938,Parlament,/parlament/plenum/abstimmung/abstimmung?id=938,['https://dserver.bundestag.de/btd/20/141/2014...,['277683']
2,2,2024-10-18,Änderungsantrag zur „Verbesserung der inneren ...,929,Inneres,/parlament/plenum/abstimmung/abstimmung?id=929,['https://dserver.bundestag.de/btd/20/128/2012...,"['275840', '276659', '276673']"
3,3,2024-10-18,Artikel 5 des Entwurfs „Verbesserung der inner...,930,Inneres,/parlament/plenum/abstimmung/abstimmung?id=930,['https://dserver.bundestag.de/btd/20/128/2012...,"['275840', '276659']"
4,4,2024-10-18,Gesetzentwurf „Verbesserung der Terrorismusbek...,931,Inneres,/parlament/plenum/abstimmung/abstimmung?id=931,['https://dserver.bundestag.de/btd/20/128/2012...,"['275841', '276659']"


In [88]:
print("CSV: drucksachen_details.csv")
df_drucksachen_details = pd.read_csv(DRUCKSACHE_PATH)
df_drucksachen_details.head()

CSV: drucksachen_details.csv


,referenznummer,docNr,docTyp,docText
0,276076.0,20/12977,Gesetzentwurf,Deutscher Bundestag Drucksache 20/12977 \n20. ...
1,277864.0,20/14302,Beschlussempfehlung und Bericht,Deutscher Bundestag Drucksache 20/14302 \n20. ...
2,277683.0,20/14150,Antrag,Deutscher Bundestag Drucksache 20/14150 \n20. ...
3,275840.0,20/12805,Gesetzentwurf,Deutscher Bundestag Drucksache 20/12805 \n20. ...
4,276659.0,20/13413,Beschlussempfehlung und Bericht,Deutscher Bundestag Drucksache 20/13413 \n20. ...


### Check uniqe DocTypes

In [ ]:
def check_unique_doc_types(csv_path):
    df = pd.read_csv(csv_path)
    unique_doc_types = df['docTyp'].unique()
    
    print("Unique types:")
    for doc_type in unique_doc_types:
        print(doc_type)

In [89]:
check_unique_doc_types(DRUCKSACHE_PATH)

Unique types:
Gesetzentwurf
Beschlussempfehlung und Bericht
Antrag
Änderungsantrag
Entschließungsantrag
nan


### Extract text functions

In [95]:
def print_extracted_text(df, referenznummer, extractFunction):
    row = df[df['referenznummer'] == referenznummer].iloc[0]
    doc_text = row['docText']
    extracted_text = extractFunction(doc_text)
    print(extracted_text)

##### Gesetzentwurf

In [92]:
def extractGesetzentwurf(doc_text):
    problem_section = re.search(r'A\. Problem und Ziel(.*?)(?=B\. Lösung|$)', doc_text, re.DOTALL)
    loesung_section = re.search(r'B\. Lösung(.*?)(?=C\. Alternativen|$)', doc_text, re.DOTALL)
    
    if problem_section and loesung_section:
        problem_text = problem_section.group(1).strip()
        loesung_text = loesung_section.group(1).strip()
        
        extracted_text = f"{problem_text} [SEP] {loesung_text}"
        return extracted_text
    else:
        return "NOT EXTRACTED"

In [101]:
referenznummer = 275840
print_extracted_text(df, referenznummer, extractGesetzentwurf)


Dieser Gesetzentwurf dient der Verbesserung der inneren Sicherheit und des 
Asylsystems.  
Der islamistische Anschlag am 23. August 2024 auf einem Volksfest in Solingen 
hat zuletzt deutlich gemacht, dass die Sicherheit im öffentlichen Raum bedroht 
ist. Die Gefährdungslage durch islamistischen Terrorismus ist anhaltend hoch und 
hat sich auch im Zuge der aktuellen Entwicklungen im Nahen Osten als Folge der 
Terroranschläge gegen den Staat Israel vom 7. Oktober 2023 weiter verschärft. 
Die extremistische Bedrohung ist nicht auf den Islamismus beschränkt. Gerade 
auch der Rechtsextremismus und Rechtsterrorismus stellen ununterbrochen eine 
große Bedrohung für unser demokratisches Gemeinweisen in Deutschland dar.  
Mit dem Sicherheitspaket nach Solingen zieht die Regierungskoalition der
Fraktionen SPD, BÜNDNIS 90/DIE GRÜNEN und FDP die nötigen Folgerungen aus 
dem Anschlag. Das betrifft insbesondere drei Bereiche: Waffenrecht,
Extremismus- und Terrorismusbekämpfung, Aufenthaltsrecht. Die

##### Beschlussempfehlung und Bericht

In [99]:
def extractBeschlussempfehlung(doc_text):
    problem_section = re.search(r'A\. Problem(.*?)(?=B\. Lösung|$)', doc_text, re.DOTALL)
    loesung_section = re.search(r'B\. Lösung(.*?)(?=C\. Alternativen|$)', doc_text, re.DOTALL)
 
    if problem_section and loesung_section:
        problem_text = problem_section.group(1).strip()
        loesung_text = loesung_section.group(1).strip()

        extracted_text = f"{problem_text} [SEP] {loesung_text}"
        return extracted_text
    else:
        return "NOT EXTRACTED"

In [100]:
referenznummer = 277864
print_extracted_text(df, referenznummer, extractBeschlussempfehlung)

Zu Buchstabe a)
Die den Gesetzentwurf vorlegenden Fraktionen und der Abgeordnete Stefan
Seidler weisen darauf hin, dass sich das Grundgesetz (GG) für eine starke
Verfassungsgerichtsbarkeit entschieden habe. Diese zentrale Weichenstellung lasse sich 
an den – auch im internationalen Vergleich – weitreichenden
Entscheidungsbefugnissen ablesen, die das GG dem Bundesverfassungsgericht zuweise. Auf
dieser Grundlage habe sich das Bundesverfassungsgericht als Garant der
freiheitlichdemokratischen Ordnung und als für Staat und Gesellschaft wesentliches
Verfassungsorgan mittlerweile fest etabliert. 
Als das GG am 24. Mai 1949 in Kraft getreten ist, sei die neuartige Institution 
„Bundesverfassungsgericht“ verfassungsrechtlich nur teilweise näher ausgeformt 
gewesen. Auch das Verständnis des Gerichts als Verfassungsorgan sei anfangs 
V
orabfassung - w
ird durch die lektorierte Fassung ersetzt.
noch nicht allgemein konsentiert gewesen. Es sei in der Folge sehr weitgehend 
am einfachen Gesetzgeber

##### Antrag

In [63]:
def extractAntrag(doc_text):
    beschluss_section = re.search(r'Der Bundestag wolle beschließen:(.*?)(?=II\.)', doc_text, re.DOTALL)
    if beschluss_section:
        beschluss_text = beschluss_section.group(1).strip()
        return beschluss_text
    else:
        return "NOT EXTRACTED"

In [102]:
referenznummer = 277683
print_extracted_text(df, referenznummer, extractAntrag)

Sections not found


##### Änderungsantrag

In [64]:
def extract_aenderungsantrag(doc_text):
    beschliessen_section = re.search(r'Der Bundestag wolle beschließen:(.*?)(?=Vorabfassung|$)', doc_text, re.DOTALL)

    if beschliessen_section:
        beschliessen_text = beschliessen_section.group(1).strip()
        return beschliessen_text
    else:
        return "NOT EXTRACTED"

In [ ]:
referenznummer = 277683
print_extracted_text(df, referenznummer, extract_aenderungsantrag)

##### Entschließungsantrag

In [65]:
def extract_entschließungsantrag(doc_text):
    abschnitt_1 = re.search(r'Der Bundestag wolle beschließen:\s*I\.(.*?)II\.', doc_text, re.DOTALL)
    abschnitt_2 = re.search(r'II\.(.*?)$', doc_text, re.DOTALL)

    if abschnitt_1 and abschnitt_2:
        abschnitt_1_text = abschnitt_1.group(1).strip()
        abschnitt_2_text = abschnitt_2.group(1).strip()
        return abschnitt_1_text + " [SEP] " + abschnitt_2_text
    else:
        return "NOT EXTRACTED"

In [ ]:
referenznummer = 277683
print_extracted_text(df, referenznummer, extract_entschließungsantrag)

### Check count of NOT EXTRACTED

In [81]:
df = pd.read_csv(DRUCKSACHE_WITH_DESC_PATH)
df.head()

,referenznummer,docNr,docTyp,docText,extractedText
0,276076.0,20/12977,Gesetzentwurf,Deutscher Bundestag Drucksache 20/12977 \n20. ...,Das Grundgesetz hat sich für eine starke Verfa...
1,277864.0,20/14302,Beschlussempfehlung und Bericht,Deutscher Bundestag Drucksache 20/14302 \n20. ...,Zu Buchstabe a)\nDie den Gesetzentwurf vorlege...
2,277683.0,20/14150,Antrag,Deutscher Bundestag Drucksache 20/14150 \n20. ...,Sections not found
3,275840.0,20/12805,Gesetzentwurf,Deutscher Bundestag Drucksache 20/12805 \n20. ...,Dieser Gesetzentwurf dient der Verbesserung de...
4,276659.0,20/13413,Beschlussempfehlung und Bericht,Deutscher Bundestag Drucksache 20/13413 \n20. ...,Zu Buchstabe a) \nDieser Gesetzentwurf soll na...


In [82]:
count_not_extracted = (df['extractedText'] == 'NOT EXTRACTED').sum()
print(f"Count EXTRACTED' in 'extractedText': {count_not_extracted}")

Count EXTRACTED' in 'extractedText': 0


### Extract all

In [66]:
def process_drucksachen_details(csv_path, output_path, samples=None):
    df = pd.read_csv(csv_path)
    
    if samples is not None:
        df = df.head(samples)
    
    df['extractedText'] = None
    
    for i, row in df.iterrows():
        doc_type = row['docTyp']
        doc_text = row['docText']
        
        if doc_type == "Gesetzentwurf":
            df.at[i, 'extractedText'] = extractGesetzentwurf(doc_text)
        elif doc_type == "Antrag":
            df.at[i, 'extractedText'] = extractAntrag(doc_text)
        elif doc_type == "Beschlussempfehlung und Bericht":
            df.at[i, 'extractedText'] = extractBeschlussempfehlung(doc_text)
        elif doc_type == "Änderungsantrag":
            df.at[i, 'extractedText'] = extract_aenderungsantrag(doc_text)
        elif doc_type == "Entschließungsantrag":
            df.at[i, 'extractedText'] = extract_entschließungsantrag(doc_text)
        else:
            df.at[i, 'extractedText'] = "NOT EXTRACTED"
    
    df.to_csv(output_path, index=False)


In [50]:
process_drucksachen_details(DRUCKSACHE_PATH, DRUCKSACHE_WITH_DESC_PATH, samples=30)